# AI/BI Genie: Natural Language Querying
## Databricks Zero to Hero - Session 47

**Objective:**
In this session, we explore **Databricks AI/BI Genie**, a tool that allows business users to interact with data using natural language (text prompts). Unlike standard dashboards, Genie allows for ad-hoc exploration without writing SQL.

**What we will cover:**
1.  Understanding Genie Spaces.
2.  **Metadata Enrichment:** Preparing data descriptions for the AI.
3.  Creating and Configuring a Genie Space.
4.  Adding **Instructions** and **Trusted Assets**.
5.  Testing with Natural Language Queries.
6.  Managing Permissions for Business Users.

**Pre-requisites:**
*   Tables `orders_raw` and `customer_raw` in your Unity Catalog (created in previous sessions).
*   A SQL Warehouse (Pro or Serverless is required for Genie).

In [ ]:
# Check if our datasets exist from the previous dashboard session
%sql
SHOW TABLES IN dev.bronze;

## 1. Metadata Enrichment (Crucial for Genie)

The AI Assistant in Genie relies heavily on **metadata** to understand your data. It looks at:
1.  Table Names
2.  Column Names
3.  **Comments/Descriptions**

If your columns have cryptic names or codes (e.g., 'O', 'F', 'P'), the AI won't know what they mean unless you define them in the comments.

Let's programmatically update the comments for `orderstatus` to ensure Genie understands the codes.

In [ ]:
-- Updating column comments so Genie understands the data dictionary
-- This avoids the need to explain it manually in the UI every time

ALTER TABLE dev.bronze.orders_raw
ALTER COLUMN o_orderstatus COMMENT 'Indicates the current status of the order. P stands for Pending, F stands for Fulfilled, O stands for Open.';

ALTER TABLE dev.bronze.orders_raw
ALTER COLUMN o_totalprice COMMENT 'Reflects the total monetary value of the order, including tax.';

-- Check the updated comments
DESCRIBE EXTENDED dev.bronze.orders_raw;

## 2. Creating the Genie Space (UI Steps)

Since Genie is a UI feature, follow these steps in the Databricks interface:

1.  **Navigate:** Click "New" -> "Genie Space".
2.  **Setup:**
    *   **Title:** Sales Genie Space
    *   **Warehouse:** Select your Serverless or Pro SQL Warehouse.
    *   **Data:** Click "Add" and select `orders_raw` and `customer_raw` from the `dev.bronze` schema.

### 3. Configuring Instructions
To help the AI handle specific business acronyms or logic, we use **General Instructions**.

**Go to Settings -> General Instructions and add:**
> "MK stands for Customer Market Segment"

This allows users to type "Show me sales by MK" and the AI will know to map "MK" to the `c_mktsegment` column.

## 3. Querying with Natural Language

Here are the queries we executed to test the Genie Space's intelligence:

| Difficulty | Natural Language Prompt | What Genie did (SQL Logic) |
| :--- | :--- | :--- |
| **Simple** | *"How many open orders?"* | Filtered `o_orderstatus = 'O'` (learned from our comment) and counted rows. |
| **Medium** | *"What is average order size based on MK?"* | Used the instruction to map MK to `c_mktsegment`, joined tables, and calculated average. |
| **Complex** | *"Year on Year growth based on Customer Market Segment"* | Generated a complex Window function query to calculate YoY percentages by segment. |

**Note:** You can always click **"Show Code"** in Genie to verify the generated SQL.

## 4. Governance and Permissions

For a business user to use this Genie Space, they need:
1.  Access to the **Genie Space** itself (Runner/Viewer access).
2.  **Unity Catalog Permissions** on the underlying data.
3.  **Usage** permissions on the SQL Warehouse.

Let's grant the necessary Unity Catalog permissions via SQL to the `business_user_grp`.

In [ ]:
-- 1. Grant usage on the Catalog and Schema
GRANT USE CATALOG ON CATALOG dev TO `business_user_grp`;
GRANT USE SCHEMA ON SCHEMA dev.bronze TO `business_user_grp`;

-- 2. Grant Select access on the specific tables used in Genie
GRANT SELECT ON TABLE dev.bronze.orders_raw TO `business_user_grp`;
GRANT SELECT ON TABLE dev.bronze.customer_raw TO `business_user_grp`;

-- 3. (Optional) If you haven't granted Warehouse access via UI, you would do it here
-- Note: Warehouse permissions are usually managed at the workspace level or via UI.

## 5. Monitoring & Feedback

*   **Monitoring:** As a creator, check the "Monitoring" tab in the Genie Space to see what questions users are asking.
*   **Feedback:** Always use the Thumbs Up/Down feature on answers. If Genie gets it wrong, provide the correct SQL. This teaches the AI ("Few-Shot Learning") for future queries.

In [ ]:
print("Genie Space configuration prerequisites completed.")
print("Permissions granted to business users.")
print("Go to the Genie UI to share the link with your stakeholders!")